# Estima a captura por categoria da zona de influência

In [1]:
import pandas as pd

In [2]:
matriz_od_horus = pd.read_csv('data/csv/matriz_od_horus.csv', delimiter = ',',low_memory=False)

In [3]:
cat_influencia = pd.read_excel('data/excel/xlsx/cat_influencia_geral.xlsx', engine='openpyxl')

In [4]:
municipios_horus = pd.read_excel('data/excel/xlsx/Lista de municipios e UTPs.xlsx', engine='openpyxl')

In [5]:
municipios_horus.columns

Index(['idMunicipio', 'codigo', 'nome', 'uf', 'idUtp', 'utp'], dtype='object')

In [6]:
def simplificar_matriz_od(df, municipios_horus):
    # Deleta a coluna 'tag' se existir
    if 'tag' in df.columns:
        df = df.drop(columns=['tag'])
    
    # Filtra para manter apenas 'modotransporte' igual a 1
    df = df[df['modotransporte'] == 1]
    
    # Mantém as colunas relevantes
    df = df[['idmunicipioorigem', 'municipioorigem', 'aerodromoembarque', 
             'aerodromodesembarque', 'idmunicipiodestino', 'municipiodestino', 'quantidadeviagem', 'mes']]
    
    # Adiciona a coluna 'cod_mun_origem' baseado na correspondência entre 'idmunicipioorigem' e 'idMunicipio'
    df = df.merge(municipios_horus[['idMunicipio', 'codigo']], left_on='idmunicipioorigem', right_on='idMunicipio', 
                  how='left').drop(columns=['idMunicipio'])
    df.rename(columns={'codigo': 'cod_mun_origem'}, inplace=True)
    
    # Adiciona a coluna 'cod_mun_destino' baseado na correspondência entre 'idmunicipiodestino' e 'idMunicipio'
    df = df.merge(municipios_horus[['idMunicipio', 'codigo']], left_on='idmunicipiodestino', right_on='idMunicipio', 
                  how='left').drop(columns=['idMunicipio'])
    df.rename(columns={'codigo': 'cod_mun_destino'}, inplace=True)
    
    # Deleta as colunas 'idmunicipioorigem', 'municipioorigem', 'idmunicipiodestino' e 'municipiodestino'
    df = df.drop(columns=['idmunicipioorigem', 'municipioorigem', 'idmunicipiodestino', 'municipiodestino'])
    
    # Agrupa por 'mes' somando os valores de 'quantidadeviagem'
    df_simplificada = df.groupby(['cod_mun_origem', 'aerodromoembarque', 'aerodromodesembarque', 
                                  'cod_mun_destino'], as_index=False)['quantidadeviagem'].sum()
    
    return df_simplificada

matriz_od_horus_simplificada = simplificar_matriz_od(matriz_od_horus, municipios_horus)

In [22]:
def criar_captura_geral(matriz_od_horus_simplificada, cat_influencia):
    # Criação de captura_origem
    captura_origem = matriz_od_horus_simplificada[['cod_mun_origem', 'aerodromoembarque', 'quantidadeviagem']]
    captura_origem = captura_origem.rename(columns={
        'cod_mun_origem': 'cod_mun', 
        'aerodromoembarque': 'aerodromo', 
        'quantidadeviagem': 'quantidadeviagem'
    })
    
    # Criação de captura_destino
    captura_destino = matriz_od_horus_simplificada[['cod_mun_destino', 'aerodromodesembarque', 'quantidadeviagem']]
    captura_destino = captura_destino.rename(columns={
        'cod_mun_destino': 'cod_mun', 
        'aerodromodesembarque': 'aerodromo', 
        'quantidadeviagem': 'quantidadeviagem'
    })
    
    # Concatenando os dois DataFrames
    captura_geral = pd.concat([captura_origem, captura_destino])
    
    # Agregando por cod_mun e aerodromo, somando a quantidade de viagens
    captura_geral = captura_geral.groupby(['cod_mun', 'aerodromo'], as_index=False)['quantidadeviagem'].sum()
    
    # Adicionando a coluna 'categoria' a captura_geral com base na correspondência em cat_influencia
    captura_geral['categoria'] = captura_geral.apply(
        lambda row: _obter_categoria(row['cod_mun'], row['aerodromo'], cat_influencia), axis=1
    )

    # Calculando a absorcao_aerodromo
    # Soma total de quantidadeviagem por cod_mun
    total_por_cod_mun = captura_geral.groupby('cod_mun')['quantidadeviagem'].transform('sum')
    
    # Calculando o percentual
    captura_geral['absorcao_aerodromo'] = (captura_geral['quantidadeviagem'] / total_por_cod_mun) * 100

    # Adicionando a coluna mun_aero com o número de repetições de cod_mun
    captura_geral['mun_aero'] = captura_geral.groupby('cod_mun')['cod_mun'].transform('count')
    
    return captura_geral

def _obter_categoria(cod_mun, aerodromo, cat_influencia):
    # Convertendo o código do aeródromo para os 4 primeiros dígitos
    prefixo_aerodromo = str(aerodromo)
    
    # Procurando a linha correspondente no DataFrame cat_influencia
    linha_cat = cat_influencia[cat_influencia['CD_MUN'] == cod_mun]
    
    if not linha_cat.empty:
        # Verificando se o prefixo do aeródromo corresponde a alguma coluna
        for coluna in linha_cat.columns[3:]:  # As colunas a partir da 4ª
            if coluna[:4] == prefixo_aerodromo:  # Comparando os 4 primeiros dígitos
                # Retorna o valor da célula correspondente à linha e coluna
                return linha_cat[coluna].values[0]
    
    # Se não encontrar, retorna um valor padrão (ex: None)
    return None

# Exemplo de chamada da função
captura_geral = criar_captura_geral(matriz_od_horus_simplificada, cat_influencia)

In [23]:
def calcular_percentual_sem_correspondencia(captura_geral, categoria_col='categoria', aerodromo_col='aerodromo', quantidadeviagem_col='quantidadeviagem'):
    # Identificar aeródromos sem correspondência
    valores_unicos = captura_geral.loc[captura_geral[categoria_col].isna(), aerodromo_col].unique()
    
    # Agrupar e somar quantidade de viagens por aeródromo
    nova_base = captura_geral[[aerodromo_col, quantidadeviagem_col]].groupby(aerodromo_col, as_index=False).sum()
    total_geral = nova_base[quantidadeviagem_col].sum()
    
    # Filtrar para aeródromos sem correspondência
    nova_base_filtrada = nova_base[nova_base[aerodromo_col].isin(valores_unicos)]
    total_filtrado = nova_base_filtrada[quantidadeviagem_col].sum()
    
    # Calcular o percentual
    percentual = (total_filtrado / total_geral) * 100
    return percentual

percentual = calcular_percentual_sem_correspondencia(captura_geral)
print(f"Os aeródromos sem correspondência representam {percentual:.2f}% do total de viagens.")

Os aeródromos sem correspondência representam 1.90% do total de viagens.


In [47]:
def criar_cat_absorcao(captura_geral):    
    # Filtrando valores onde categoria não é NaN
    captura_filtrada = captura_geral[captura_geral['categoria'].notna()]
    
    # Aplicando o filtro adicional
    categorias_excluidas = ['G', 'H', 'I', 'J']
    captura_filtrada = captura_filtrada[
        ~((captura_filtrada['mun_aero'] == 1) & (captura_filtrada['categoria'].isin(categorias_excluidas)))
    ]
    
    # Agrupando por categoria e calculando a média de absorcao_aerodromo
    cat_absorcao = captura_filtrada.groupby('categoria', as_index=False).agg(
        absorcao=('absorcao_aerodromo', 'mean')
    )
    
    return cat_absorcao

cat_absorcao = criar_cat_absorcao(captura_geral)

In [49]:
cat_absorcao.to_excel('data/excel/xlsx/cat_absorcao_nacional.xlsx', index = False)